In [10]:
#@title Mount google drive and Install dependencies.

from google.colab import drive
drive.mount('/content/drive')

!pip uninstall -y bitsandbytes

!pip install -q diffusers==0.27.2
!pip install -q transformers==4.38.2 accelerate==0.29.1 safetensors==0.4.2
!pip install huggingface_hub==0.20.3
!pip install peft==0.7.0
!accelerate config default

import os
import random
import torch
import torch._dynamo
from torch import autocast
torch._dynamo.config.suppress_errors = True
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found existing installation: bitsandbytes 0.43.0
Uninstalling bitsandbytes-0.43.0:
  Successfully uninstalled bitsandbytes-0.43.0
Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
[W1229 04:55:34.787731892 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())


In [1]:
#@title Mode Path and Images Path

#@markdown Enter the google drive path to save generated images.
import os


lora_path = "/content/drive/MyDrive/ACV_Final/model/checkpoint-800"#@param {type:"string"}
output_path = "/content/drive/MyDrive/ACV_Final/output" #@param {type:"string"}

# 检查文件夹是否存在，如果不存在，则创建它
if not os.path.exists(output_path):
    os.makedirs(output_path)
print(f"[*] images will be saved at {output_path}")

[*] images will be saved at /content/drive/MyDrive/ACV_Final/output


In [2]:
#@title Setup Pipeline

from diffusers import DiffusionPipeline, EulerDiscreteScheduler
import torch
import numpy as np
import cv2
from PIL import Image
from huggingface_hub import hf_hub_download

# Initialize ControlNet pipeline.
pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True)

pipeline.load_lora_weights(lora_path, weight_name = 'pytorch_lora_weights.safetensors', local_files_only=True, adapter_name="lora_2")
pipeline.load_lora_weights(hf_hub_download("ByteDance/SDXL-Lightning", "sdxl_lightning_4step_lora.safetensors"), adapter_name="lora_1")

pipeline.set_adapters(["lora_1", "lora_2"], adapter_weights=[1.0, 1.0])

# Ensure sampler uses "trailing" timesteps.
pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config, timestep_spacing="trailing")
pipeline.fuse_lora()

pipeline = pipeline.to("cuda")

pipeline.enable_model_cpu_offload()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

sdxl_lightning_4step_lora.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

In [3]:
#@title Define Functions
import random

# Define Function for generating and refining images.

from PIL import Image

def generate(prompt, negative_prompt, guidance_scale, num_inference_steps, height, width, image_number_for_each_prompt, save_images_Path):

  # Define the folder path to save the images
  save_folder = save_images_Path
  # Create the folder if it doesn't exist
  os.makedirs(save_folder, exist_ok=True)

  seeds = [random.randint(1000000000000, 9999999999999) for _ in range(image_number_for_each_prompt)]

  for seed in seeds:
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipeline(prompt=prompt,
                 negative_prompt=negative_prompt,
                 generator=generator,
                 height=height,
                 width=width,
                 num_images_per_prompt=1,
                 num_inference_steps=num_inference_steps,
                 guidance_scale=guidance_scale).images[0]
    image_path = os.path.join(save_folder, f"{prompt}_{seed}.jpg")
    image.save(image_path)

    torch.cuda.empty_cache()
  return

In [7]:
prompt_list = [
               "ACV style of photo,  boat from left",
               "ACV style of photo, boat from right ",
               "ACV style of photo, boat from right sideview ",
               "ACV style of photo, boat from left sideview ",
               "ACV style of photo, boat from bridview ",

              ]

In [8]:
negative_prompt = "blur, bad composition, bad anatomy, anime, cartoon, graphic, text, painting, crayon, graphite, abstract, concept, low resolution"

In [ ]:
#@title Inference Parameters

guidance_scale = 0 #@param {type:"integer"}
num_inference_steps = 4 #@param {type:"integer"}
height = 512 #@param {type:"integer"}
width = 512 #@param {type:"integer"}
image_number_for_each_prompt = 4 #@param {type:"integer"}


for prompt in prompt_list:
  generate(prompt, negative_prompt, guidance_scale, num_inference_steps, height, width, image_number_for_each_prompt, output_path)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#@title End Runtime

from google.colab import runtime

runtime.unassign()